In [1]:
import findspark
findspark.init()

In [21]:
channels_not_to_consider = [#traffic
        '511NY', '511NYC', 'TotalTrafficNYC', '511nyNJ', '511NYMidHudson',
       'Reported_NYC', '511ny456', '511nyAlbany', '511nyACE', '511ny123', 
        '511nyBDFV', '511nyWNY', '511nyRochester',
       '511nyLongIsland', 'AllKindsWeather', 
       '511ny7',
    #jobs
    'tmj_nyc_jobs', 'tmj_nyc_adv', 'CalvaryHospJobs',
       'tmj_RAM_cstsrv', 'tmj_nyc_mgmt', 'tmj_roc_cler',
       'tmj_NAS_edu', 'tmj_nyc_it', 'tmj_NYC_schn', 'tmj_NJN_cstsrv',
       'tmj_nwk_retail', 'tmj_nyc_nursing', 'tmj_nyc_sales', 'tmj_nyc_cler',
       'USSJobs', 'tmj_nwk_sales', 'tmj_nyc_legal', 'tmj_nwk_eng',
       'tmj_nyc_retail', 'tmj_nyc_transp', 'WGPNursingJobs', 'tmj_nwk_socsci',
       'tmj_NYC_adm', 'nwkmeddevice', 'tmj_nyc_edu', 'tmj_nwk_secure',
       'nwknursing',
    'tmj_nyc_finance', 'tmj_NYC_secure', 'tmj_nyc_acct', 'tmj_nwk_cler',
       'tmj_nyc_banking', 'tmj_ny_hrta', 'tmj_nwk_mgmt', 'tmj_nwk_auto',
       'tmj_nyc_cstsrv', 'tmj_nyc_health', 'nwkhealth', 'tmj_nwk_cstsrv',
       'tmj_nyc_manuf', 'tmj_cte_nursing', 'tmj_nyc_eng', 'tmj_nyc_itpm1',
       'tmj_nyc_hr', 'tmj_NY_sales', 'CVSHealthJobs', 'tmj_NAS_mgmt',
       'CompassJobBoard', 'tmj_nyc_hrta', 'tmj_NAS_nursing', 'BostonMarketJob',
       'tmj_nyc_labor', 'MetsAvenue', 'tmj_nwk_schn', 'tmj_RAM_nursing',
       'tmj_nwk_acct', 'tmj_nwk_jobs', 'tmj_nyc_art', 'tmj_nwk_labor',
       'tmj_roc_eng', 'tmj_NAS_transp', 'CA_ROC_Jobs2', 'tmj_nyc_cosmo',
       'tmj_RAM_edu', 'tmj_NAS_health', 'tmj_nwk_facmgmt', 'tmj_NAS_facmgmt',
       'GodivaJobs', 'tmj_RAM_acct', 'tmj_roc_health', 'tmj_nyc_itdb',
       'tmj_nwk_transp', 'tmj_nwk_edu', 'tmj_RAM_retail', 'tmj_RAM_mgmt',
       'tmj_NAS_socsci', 'tmj_nwk_prod', 'tmj_nyc_realest', 'tmj_NJ_facmgmt', 'tmj_njn_retail',
       'tmj_roc_nursing', 'tmj_nwk_finance', 'Fly_Sistah', 'tmj_NYC_skltrd',
       'tmj_nya_nursing', 'tmj_nwk_web', 'tmj_roc_cstsrv', 'tmj_nys_jobs',
       'tmj_njc_hrta', 'tmj_NAS_retail', 'tmj_roc_hrta',
       'ChurchCathy', 'tmj_NAS_secure', 'tmj_RAM_art', 'tmj_NAS_labor',
       'tmj_NAS_physici', 'tmj_nwk_skltrd', 'tmj_roc_sales', 'tmj_nwk_purch',
       'tmj_NYS_NURSING', 'tmj_nwk_physici', 'tmj_njn_hrta', 'Mezikenyc',
       'JCI_Jobs', 'tmj_NAS_acct', 'tmj_NYC_gensci', 'tmj_nya_eng',
       'tmj_nwk_nonprft', 'tmj_roc_manuf', 'nwkitsupport', 'tmj_NY_LABOR',
       'tmj_ny_mgmt', 'tmj_njn_health',
       'tmj_nj_hrta', 'tmj_NAS_cstsrv', 'tmj_nwk_it', 'tmj_nya_transp',
       'tmj_ct_nursing', 'tmj_NJ_sales', 'tmj_nya_acct',
       'nwkmanuf', 'tmj_nys_cstsrv', 'tmj_njn_nursing',
       'tmj_njn_mgmt', 'cbwaszak', 'tmj_NAS_cler', 'tmj_RAM_auto',
       'tmj_nwk_art'
    'WWEWomenMatter',
    #photos
    'ThomGambino', 'Xsanthemum', 'francesco212', 'Empressjurnee',
       'andrerivera801', 'janice830', 'Ingridebap', 'StevieSoFetch_',
       'EstebanDaHost', 'graceyhanderson', 'bccdny', 'brian_wood_'
       ]

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("TweetsLDA") \
    .getOrCreate()

In [4]:
sc = spark.sparkContext

### Reading CSV

In [5]:
df = spark.read.csv('../get-tweets-by-geolocation/data/usa_training_tweets_23_06.csv', header=True, sep=',')

In [9]:
filtered_df = df.rdd.filter(lambda x: filter_tweet(x[0]))

In [10]:
filtered_df.count()

1177561

In [50]:
df.rdd.count()

1292100

In [7]:
sample = sc.parallelize(df.take(10))

### Text preprocessing 

In [16]:
import re
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
tokenizer = nltk.WordPunctTokenizer()
lemmatizer = WordNetLemmatizer()
stop_word_list = nltk.corpus.stopwords.words('english')

In [18]:
def filter_tweet(tweet):
    
    if not isinstance(tweet, str):
        is_filtered = True
    elif len(tweet.split(' ')) < 3:
        is_filtered = True
    else: 
        is_filtered = False
        
    return not is_filtered
         
def process_tweet(tweet):
   
    tweet = tweet.lower() # get lowercase
    tweet = re.sub(r'@\w+', '', tweet) # filter words with non-letters at the beginning (mainly for mentions)
    tweet = re.sub(r'http://\S{,280}', '', tweet) # filter http
    tweet = re.sub(r'https://\S{,280}', '', tweet) # filter https
    tweet = re.sub(r'[^A-Za-z]', ' ', tweet) # filter all non-letters
    tweet = re.sub(r'\s{2,}', ' ', tweet) # remove multiply whitespaces
    tweet = re.sub(r'(.)\1{2,}', r'\1', tweet) # remove repeated chars (e.g. "greeeeat" -> "great")
    tweet = tweet.strip() # remove possible whitespaces from both sides of the tweet

    # lemmatize, tokenize and conquer
    processed_tweet = [lemmatizer.lemmatize(token) for token in tokenizer.tokenize(tweet)
                       if token not in stop_word_list]
    
    return processed_tweet

In [19]:
processed_df = filtered_df.map(lambda x: process_tweet(x[0]))
processed_df = processed_df.filter(lambda x: len(x) > 0)

In [ ]:
df = df.filter(lambda x: x)

In [40]:
sample.map(lambda x: process_tweet(x[0])).collect()

[['twin'],
 ['tomorrow',
  'come',
  'join',
  'u',
  'ice',
  'cream',
  'swirll',
  'cafe',
  'located',
  'cedar',
  'ave',
  'facing',
  'baltimore',
  'ave',
  'look',
  'forwa'],
 ['know'],
 ['greatest', 'story', 'ever', 'told', 'amen'],
 ['hahaha', 'bet', 'island', 'landed', 'hopping', 'tho'],
 ['believe', 'stole', 'tweet', 'smh'],
 ['new', 'starting', 'quarterback'],
 ['going', 'must', 'come', 'wall', 'talking', 'bout'],
 ['yep',
  'pop',
  'culture',
  'standpoint',
  'recognized',
  'depicted',
  'planet',
  'due',
  'ring'],
 ['protect', 'cost', 'national', 'treasure']]

In [46]:
sample.map(lambda x: process_tweet(x[0])).collect()

[['twin', '\U0001f974'],
 ['tomorrow',
  'come',
  'join',
  'u',
  'ice',
  'cream',
  '🍦',
  'swirll',
  'cafe',
  'located',
  'cedar',
  'ave',
  'facing',
  'baltimore',
  'ave',
  'look',
  'forwa'],
 ['know', '🙌🏽'],
 ['livingforjc', 'greatest', '❤️', 'story', 'ever', 'told', 'amen'],
 ['pastor', 'coat', 'hahaha', 'bet', 'island', 'landed', 'hopping', 'tho'],
 ['uzsagi', 'cant', 'believe', 'stole', 'tweet', 'smh'],
 ['treenkarli', 'new', 'starting', 'quarterback'],
 ['dtill', 'abc', 'going', 'must', 'come', 'wall', 'talking', 'bout'],
 ['killswitch',
  'yep',
  'pop',
  'culture',
  'standpoint',
  'recognized',
  'depicted',
  'planet',
  'due',
  'ring'],
 ['protect', 'loganmania', 'cost', 'national', 'treasure']]

In [45]:
sample.map(lambda x: x[0]).collect()

['Twins \U0001f974',
 'Tomorrow come join us over ice cream 🍦 at Swirll Cafe located at 4610 Cedar Ave facing Baltimore Ave! We look forwa… https://t.co/h6UiREy5at',
 'You know what 🙌🏽',
 '@livingforjc The greatest ❤️ story ever told amen',
 '@268pastor @_coat Hahaha I bet that island they landed on is hopping now tho',
 "@uzsagi I can't believe they stole your tweet smh",
 '@TreenKarli the new starting quarterback',
 '@dtill96 @ABC What going up Must come down same as that wall you talking bout.',
 '@kill_switch_0 Yep. and just from a pop culture standpoint, it’s the most recognized and most depicted planet due to its rings',
 'Protect @LoganMania at all costs. He is a national treasure. ']